# Spatial-Join Notebook

En este notebook exploramos la posibilidad de asignar cada instalación de monitorización de ruido del ayuntamiento de Barcelona a un tramo de tránsito mediante la técnica de Spatial-Join.

In [1]:
import os
import pathlib
import pandas as pd

In [11]:
tramos_filepath = pathlib.Path(os.getcwd()) / "transit_relacio_trams.csv"
df_tramos = pd.read_csv(tramos_filepath)
df_tramos.head()

,Tram,Descripció,Coordenades
0,1,Diagonal (Ronda de Dalt a Doctor Marañón),"2.11203535639414,41.3841912394771,2.1015028628..."
1,2,Diagonal (Doctor Marañón a Ronda de Dalt),"2.111944376806616,41.38446666680338,2.10159408..."
2,3,Diagonal (Doctor Marañón a Pl. Pius XII),"2.112093343037027,41.38422850920645,2.12264979..."
3,4,Diagonal (Pl. Pius XII a Doctor Marañón),"2.122592049318304,41.38719094189204,2.11196902..."
4,5,Diagonal (Pl. Pius XII a Pl. Maria Cristina),"2.122657659295115,41.38694195794678,2.12755961..."


In [12]:
from shapely import LineString
import geopandas as gpd

# Convertir coordenadas de tramos a LineString
# La columna 'Coordenades' tiene el formato 'x1,y1,x2,y2'
df_tramos['geometry'] = df_tramos['Coordenades'].apply(
    lambda x: LineString([(float(x.split(',')[0]), float(x.split(',')[1])), 
               (float(x.split(',')[2]), float(x.split(',')[3]))])
)

gdf_tramos = gpd.GeoDataFrame(df_tramos, geometry='geometry')

In [13]:
gdf_tramos.head()

,Tram,Descripció,Coordenades,geometry
0,1,Diagonal (Ronda de Dalt a Doctor Marañón),"2.11203535639414,41.3841912394771,2.1015028628...","LINESTRING (2.11204 41.38419, 2.1015 41.38163)"
1,2,Diagonal (Doctor Marañón a Ronda de Dalt),"2.111944376806616,41.38446666680338,2.10159408...","LINESTRING (2.11194 41.38447, 2.10159 41.38187)"
2,3,Diagonal (Doctor Marañón a Pl. Pius XII),"2.112093343037027,41.38422850920645,2.12264979...","LINESTRING (2.11209 41.38423, 2.12265 41.38693)"
3,4,Diagonal (Pl. Pius XII a Doctor Marañón),"2.122592049318304,41.38719094189204,2.11196902...","LINESTRING (2.12259 41.38719, 2.11197 41.38446)"
4,5,Diagonal (Pl. Pius XII a Pl. Maria Cristina),"2.122657659295115,41.38694195794678,2.12755961...","LINESTRING (2.12266 41.38694, 2.12756 41.38818)"


In [14]:
instalaciones_filepath = pathlib.Path(os.getcwd()) / "XarxaSoroll_EquipsMonitor_Instal.csv"
df_instalaciones = pd.read_csv(instalaciones_filepath)
df_instalaciones.head()

,Id_Instal,Codi_Carrer,Tipus_Via,Nom_Carrer,Num_Carrer,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Latitud,Longitud,ETRS89_X,ETRS89_Y,Data_Instalacio,Data_DesInstalacio,Font
0,186,348408,C,Tuset,8,26,Galvany,5,Sarrià-Sant Gervasi,41.395450,2.152753,429172.441,4583003.592,29/01/2014,16/01/2015,OCI
1,187,348408,C,Tuset,17,26,Galvany,5,Sarrià-Sant Gervasi,41.396156,2.151564,429073.823,4583082.946,29/01/2014,16/01/2015,OCI
2,206,112208,C,Escudellers,53,2,el Barri Gòtic,1,Ciutat Vella,41.380127,2.176924,431177.000,4581283.000,05/02/2014,30/01/2015,OCI
3,207,86902,C,Nou de la Rambla,21,1,el Raval,1,Ciutat Vella,41.378273,2.173348,430876.000,4581080.000,05/02/2014,30/01/2015,OCI
4,208,46409,Pg,Born,19,4,"Sant Pere, Santa Caterina i la Ribera",1,Ciutat Vella,41.384925,2.182772,431671.000,4581811.000,05/02/2014,30/01/2015,OCI


In [17]:
from shapely.geometry import Point
df_instalaciones['geometry'] = df_instalaciones.apply(lambda row: Point(float(row['Latitud']), float(row['Longitud'])), axis=1)
gdf_instalaciones = gpd.GeoDataFrame(df_instalaciones, geometry='geometry')
gdf_instalaciones.head()

,Id_Instal,Codi_Carrer,Tipus_Via,Nom_Carrer,Num_Carrer,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Latitud,Longitud,ETRS89_X,ETRS89_Y,Data_Instalacio,Data_DesInstalacio,Font,geometry
0,186,348408,C,Tuset,8,26,Galvany,5,Sarrià-Sant Gervasi,41.395450,2.152753,429172.441,4583003.592,29/01/2014,16/01/2015,OCI,POINT (41.39545 2.15275)
1,187,348408,C,Tuset,17,26,Galvany,5,Sarrià-Sant Gervasi,41.396156,2.151564,429073.823,4583082.946,29/01/2014,16/01/2015,OCI,POINT (41.39616 2.15156)
2,206,112208,C,Escudellers,53,2,el Barri Gòtic,1,Ciutat Vella,41.380127,2.176924,431177.000,4581283.000,05/02/2014,30/01/2015,OCI,POINT (41.38013 2.17692)
3,207,86902,C,Nou de la Rambla,21,1,el Raval,1,Ciutat Vella,41.378273,2.173348,430876.000,4581080.000,05/02/2014,30/01/2015,OCI,POINT (41.37827 2.17335)
4,208,46409,Pg,Born,19,4,"Sant Pere, Santa Caterina i la Ribera",1,Ciutat Vella,41.384925,2.182772,431671.000,4581811.000,05/02/2014,30/01/2015,OCI,POINT (41.38492 2.18277)


In [ ]:
# Convertir CRS a EPSG:25831, que es el CRS proyectado para España
# gdf_instalaciones.to_crs("EPSG:25831", inplace=True)
# gdf_tramos.to_crs("EPSG:25831", inplace=True)

In [18]:
joined = gdf_instalaciones.sjoin_nearest(
    gdf_tramos,
    how="left",
    max_distance=100
)

In [19]:
joined.head()

,Id_Instal,Codi_Carrer,Tipus_Via,Nom_Carrer,Num_Carrer,Codi_Barri,Nom_Barri,Codi_Districte,Nom_Districte,Latitud,...,ETRS89_X,ETRS89_Y,Data_Instalacio,Data_DesInstalacio,Font,geometry,index_right,Tram,Descripció,Coordenades
0,186,348408,C,Tuset,8,26,Galvany,5,Sarrià-Sant Gervasi,41.395450,...,429172.441,4583003.592,29/01/2014,16/01/2015,OCI,POINT (41.39545 2.15275),485,526,Ronda del Litoral (Rambla de Prim a Bac de Roda),"2.22137891826153,41.41009708737582,2.221022044..."
1,187,348408,C,Tuset,17,26,Galvany,5,Sarrià-Sant Gervasi,41.396156,...,429073.823,4583082.946,29/01/2014,16/01/2015,OCI,POINT (41.39616 2.15156),485,526,Ronda del Litoral (Rambla de Prim a Bac de Roda),"2.22137891826153,41.41009708737582,2.221022044..."
2,206,112208,C,Escudellers,53,2,el Barri Gòtic,1,Ciutat Vella,41.380127,...,431177.000,4581283.000,05/02/2014,30/01/2015,OCI,POINT (41.38013 2.17692),485,526,Ronda del Litoral (Rambla de Prim a Bac de Roda),"2.22137891826153,41.41009708737582,2.221022044..."
3,207,86902,C,Nou de la Rambla,21,1,el Raval,1,Ciutat Vella,41.378273,...,430876.000,4581080.000,05/02/2014,30/01/2015,OCI,POINT (41.37827 2.17335),485,526,Ronda del Litoral (Rambla de Prim a Bac de Roda),"2.22137891826153,41.41009708737582,2.221022044..."
4,208,46409,Pg,Born,19,4,"Sant Pere, Santa Caterina i la Ribera",1,Ciutat Vella,41.384925,...,431671.000,4581811.000,05/02/2014,30/01/2015,OCI,POINT (41.38492 2.18277),485,526,Ronda del Litoral (Rambla de Prim a Bac de Roda),"2.22137891826153,41.41009708737582,2.221022044..."


In [20]:
joined.to_csv('spatial-joined.csv', header=True, index=False)